In [ ]:
!pip install datasets wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are insta

In [1]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler
from transformers import AutoTokenizer
from datasets import load_dataset
import os
import random
# import wandb

/home/ritish/prog/ML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the Multi30k dataset
dataset = load_dataset("bentrevett/multi30k", split="train")
val_dataset = load_dataset("bentrevett/multi30k", split="validation")
test_dataset = load_dataset("bentrevett/multi30k", split="test")

In [3]:
from transformers import BertTokenizer, DataCollatorForSeq2Seq

In [4]:
# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
vocab_size = tokenizer.vocab_size

In [5]:
# Create a config for the model
config = {
    'n_embed': 256,
    'n_head': 8,
    'n_layer': 6,
    'block_size': 64,
    'dropout': 0.1,
    'batch_size': 64,
    'lr': 1,
    'tokenizer': 'bert-base-uncased',
    'model_type': 'layerwise encoder output'
}

In [ ]:
wandb.init(
    project='Transformer_multi30k_enc_final_layer',
    entity='ritishtest1',

    config=config
    )

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
# Tokenize the dataset
def tokenize_function(examples):
    source = tokenizer(examples["en"], padding="max_length", truncation=True, max_length=config['block_size'])
    target = tokenizer(examples["de"], padding="max_length", truncation=True, max_length=config['block_size'])
    source["labels"] = target["input_ids"]
    return source

In [7]:
dataset[:2]

{'en': ['Two young, White males are outside near many bushes.',
  'Several men in hard hats are operating a giant pulley system.'],
 'de': ['Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
  'Mehrere Männer mit Schutzhelmen bedienen ein Antriebsradsystem.']}

In [8]:
dataset[0]['en']

'Two young, White males are outside near many bushes.'

In [9]:
dataset['en'][:10]

['Two young, White males are outside near many bushes.',
 'Several men in hard hats are operating a giant pulley system.',
 'A little girl climbing into a wooden playhouse.',
 'A man in a blue shirt is standing on a ladder cleaning a window.',
 'Two men are at the stove preparing food.',
 'A man in green holds a guitar while the other man observes his shirt.',
 'A man is smiling at a stuffed lion',
 'A trendy girl talking on her cellphone while gliding slowly down the street.',
 'A woman with a large purse is walking by a gate.',
 'Boys dancing on poles in the middle of the night.']

In [10]:
dataset

Dataset({
    features: ['en', 'de'],
    num_rows: 29000
})

In [11]:
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["en", "de"])
tokenized_val_datasets = val_dataset.map(tokenize_function, batched=True, remove_columns=["en", "de"])
tokenized_test_datasets = test_dataset.map(tokenize_function, batched=True, remove_columns=["en", "de"])

Map: 100%|██████████| 1000/1000 [00:00<00:00, 3382.32 examples/s]


In [12]:
# Create data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=None)

# Create DataLoaders
train_dataloader = DataLoader(tokenized_datasets, shuffle=True, batch_size=config['batch_size'], collate_fn=data_collator)
val_dataloader = DataLoader(tokenized_val_datasets, shuffle=False, batch_size=config['batch_size'], collate_fn=data_collator)

In [13]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size, causal, cross=False):
        super().__init__()
        self.num_heads = num_heads
        self.head_size = head_size
        self.causal = causal
        self.cross = cross
        self.q_attn = nn.Linear(config['n_embed'], config['n_embed'], bias=False)
        self.kv_attn = nn.Linear(config['n_embed'], config['n_embed'] * 2, bias=False)
        self.c_proj = nn.Linear(config['n_embed'], config['n_embed'], bias=False)             # Combine the outputs of the heads
        self.attn_dropout = nn.Dropout(config['dropout'])
        self.resid_dropout = nn.Dropout(config['dropout'])

        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')

        ############################### REMOVE THIS #########################################
        self.flash = False

        if not self.flash:
            print("WARNING: using slow attention.")
            # self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)).view(1, 1, block_size, block_size))
            self.register_buffer("tril", torch.tril(torch.ones(config['block_size'], config['block_size'])))

    def forward(self, x, y=None, mask=None):
        B, T_dec, C = x.shape

        if y is None:
            y = x

        B, T_enc, C = y.shape

        # Query from the decoder
        q = self.q_attn(x).view(B, T_dec, self.num_heads, self.head_size).transpose(1, 2)  # (B,T_dec,C) -> (B,T_dec,nh,hs) -> (B,nh,T_dec,hs)
        k, v = self.kv_attn(y).split(C, dim=-1)                                 # (B,T_enc,2C) -> 2*(B,T_enc,C)
        k = k.view(B, T_enc, self.num_heads, self.head_size).transpose(1, 2)    # (B,nh,T_enc,hs)
        v = v.view(B, T_enc, self.num_heads, self.head_size).transpose(1, 2)    # (B,nh,T_enc,hs)

        if self.flash:
            out = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, is_causal=self.causal)
        else:
            wei = q @ k.transpose(-2, -1) * 1.0 / math.sqrt(k.size(-1))         # (B,nh,T_dec,hs) @ (B,nh,hs,T_enc) -> (B,nh,T_dec,T_enc)
            if self.causal:
                if mask is None:
                    wei = wei.masked_fill(self.tril[:T_dec, :T_enc] == 0, float('-inf')) # (B,nh,T_dec,T_enc) - lower triangular mask
                else:
                    wei = wei.masked_fill(mask == 0, float('-inf'))             # (B,nh,T_dec,T_enc) - mask for masking future tokens
            wei = F.softmax(wei, dim=-1)                                        # (B,nh,T_dec,T_enc)
            wei = self.attn_dropout(wei)
            out = wei @ v                                                       # (B,nh,T_dec,T_enc) @ (B,nh,T_enc,hs) -> (B,nh,T_dec,hs)

        out = out.transpose(1, 2).contiguous().view(B, T_dec, -1)               # (B,T_dec,nh*hs) = (B,T_dec,C)
        out = self.resid_dropout(self.c_proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(config['dropout']),
        )

    def forward(self, x):
        return self.net(x)

class DecoderBlock(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embed, n_head, cross=False):
        # n_embed: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        self.cross = cross
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size, causal=True, cross=False)
        if self.cross:
            # Cross attention block
            self.cross_sa = MultiHeadAttention(n_head, head_size, causal=False, cross=True)
        self.ffwd = FeedFoward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)
        self.ln3 = nn.LayerNorm(n_embed)

    def forward(self, x, y=None):
        x = x + self.sa(self.ln1(x))
        if self.cross:
            x = x + self.cross_sa(self.ln2(x), y)
        x = x + self.ffwd(self.ln3(x))
        return x

class EncoderBlock(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embed, n_head):
        # n_embed: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size, causal=False, cross=False)
        self.ffwd = FeedFoward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# Encoder-decoder transformer
class Transformer(nn.Module):
    def __init__(self):
        super().__init__()
        # Create token embedding table
        self.token_embedding_table_enc = nn.Embedding(vocab_size, config['n_embed'])
        self.token_embedding_table_dec = nn.Embedding(vocab_size, config['n_embed'])

        # Create encoder blocks
        self.enc_blocks = nn.ModuleList([EncoderBlock(config['n_embed'], config['n_head']) for _ in range(config['n_layer'])])

        # Create decoder blocks with masked multihead attention followed by cross attention
        self.dec_blocks = nn.ModuleList([DecoderBlock(config['n_embed'], config['n_head'], cross=True) for _ in range(config['n_layer'])])

        self.ln_f = nn.LayerNorm(config['n_embed']) # final layer norm
        self.lm_head = nn.Linear(config['n_embed'], vocab_size)

    def generate_sinousidal_pos_embeddings(self, T, device):
        pos = torch.arange(T, device=device).view(-1,1)              # (T, 1)
        i = torch.arange(config['n_embed'], device=device).view(1,-1)          # (1, C)
        angles = pos / 10000**(2*i/(config['n_embed']))                        # (T, C)
        angles[:, 0::2] = torch.sin(angles[:, 0::2])
        angles[:, 1::2] = torch.cos(angles[:, 1::2])
        return angles.unsqueeze(0)

    def forward(self, enc_inp, dec_inp, targets=None):
        B, T_enc = enc_inp.shape
        B, T_dec = dec_inp.shape

        # enc_inp and targets are both (B,T) tensor of integers
        tok_emb_enc = self.token_embedding_table_enc(enc_inp)                                  # (B,T,C)
        tok_emb_dec = self.token_embedding_table_dec(dec_inp)                                  # (B,T,C)

        # Encoder and decoder positional embeddings
        pos_emb_enc = self.generate_sinousidal_pos_embeddings(T_enc, device=enc_inp.device)        # (1,T_enc,C)
        pos_emb_dec = self.generate_sinousidal_pos_embeddings(T_dec, device=dec_inp.device)        # (1,T_dec,C)

        x_enc = tok_emb_enc + pos_emb_enc                                       # (B,T_enc,C)
        x_dec = tok_emb_dec + pos_emb_dec                                       # (B,T_dec,C)

        # Layer wise cross attention
        for i in range(config['n_layer']):
            x_enc = self.enc_blocks[i](x_enc)                                   # (B,T_enc,C)
            x_dec = self.dec_blocks[i](x_dec, x_enc)                            # (B,T_dec,C)

        x = self.ln_f(x_dec)                                                    # (B,T_dec,C)
        logits = self.lm_head(x)                                                # (B,T_dec,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape

            logits = logits.reshape(B*T, C)
            targets = targets.reshape(B*T)
            loss = F.cross_entropy(logits, targets, ignore_index=tokenizer.pad_token_id)

        return logits, loss

    def generate(self, enc_inp, max_len=config['block_size'], greedy=True):
        with torch.no_grad():
            dec_inp = 101 * torch.ones((1, 1), dtype=torch.long, device=enc_inp.device)
            enc_inp = enc_inp[:, :config['block_size']]

            for i in range(max_len-1):
                dec_inp = dec_inp[:, -config['block_size']:]
                logits, _ = self(enc_inp, dec_inp)
                if greedy:
                    next_token = torch.argmax(logits[:, -1, :], dim=-1).view(-1, 1)
                else:
                    next_token = torch.multinomial(F.softmax(logits[:, -1, :], dim=-1), num_samples=1).view(-1, 1)
                dec_inp = torch.cat([dec_inp, next_token], dim=1)
                if next_token == 102:
                    break

        return dec_inp

    def generate_v1(self, enc_inp, max_len=config['block_size'], greedy=True):
        with torch.no_grad():
            dec_inp = 101 * torch.ones((1, 1), dtype=torch.long, device=enc_inp.device)
            enc_inp = enc_inp[:, :config['block_size']]

            B, T_enc = enc_inp.shape
            x_enc = self.token_embedding_table_enc(enc_inp)
            pos_emb_enc = self.generate_sinousidal_pos_embeddings(T_enc, device=enc_inp.device)
            x_enc = x_enc + pos_emb_enc
            x_enc_out = torch.zeros((1, config['block_size'], config['n_embed'], config['n_layer'])).to(enc_inp.device)

            # Precompute the encoder outputs
            for i in range(config['n_layer']):
                x_enc = self.enc_blocks[i](x_enc)                                   # (B,T_enc,C)
                x_enc_out[:, :, :, i] = x_enc

            for i in range(max_len-1):
                dec_inp = dec_inp[:, -config['block_size']:]
                x_dec = self.token_embedding_table_dec(dec_inp)
                pos_emb_dec = self.generate_sinousidal_pos_embeddings(dec_inp.shape[1], device=dec_inp.device)
                x_dec = x_dec + pos_emb_dec

                # Layer wise cross attention
                for j in range(config['n_layer']):
                    x_dec = self.dec_blocks[j](x_dec, x_enc_out[:,:,:,j])           # (B,T_dec,C)

                x = self.ln_f(x_dec)                                                # (B,T_dec,C)
                logits = self.lm_head(x)                                            # (B,T_dec,vocab_size)

                if greedy:
                    next_token = torch.argmax(logits[:, -1, :], dim=-1).view(-1, 1)
                else:
                    next_token = torch.multinomial(F.softmax(logits[:, -1, :], dim=-1), num_samples=1).view(-1, 1)

                dec_inp = torch.cat([dec_inp, next_token], dim=1)
                if next_token == 102:
                    break

        return dec_inp

In [14]:
model = Transformer()
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')
n_epoch = 0
n_step = 0

# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cpu'
model = model.to(device)

criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

34.512698 M parameters


In [15]:
# Learning rate scheduling with warmup steps
def lr_scheduler(step, n_embed=config['n_embed'], warmup_steps=400):
    step = max(1, step)
    lr = (n_embed ** -0.5) * min(step ** -0.5, step * warmup_steps ** -1.5)
    return lr

optimizer = torch.optim.AdamW(model.parameters(), lr=config['lr'])
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_scheduler)

In [16]:
# Create a new training loop that prints train and val loss at every 100 steps
# Training loop
max_epochs = 1
for epoch in range(max_epochs):
    n_epoch += 1
    model.train()
    total_loss = 0
    for step, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)

        # Create labels for autoregressive training of decoder
        decoder_input_ids = labels[:, :-1].contiguous()
        labels = labels.contiguous()

        # Add padding to the start of decoder input
        decoder_input_ids = F.pad(decoder_input_ids, (1, 0), value=tokenizer.pad_token_id)

        # Forward pass
        logits, loss = model(input_ids, decoder_input_ids, labels)

        # Backward pass
        loss.backward()

        optimizer.step()
        scheduler.step()
        n_step += 1
        total_loss += loss.item()
        if step % 100 == 0:
            # Calculate val loss
            val_loss = 0
            model.eval()

            # Disable gradient calculation
            with torch.no_grad():
                for val_step, val_batch in enumerate(val_dataloader):
                    input_ids = val_batch["input_ids"].to(device)
                    labels = val_batch["labels"].to(device)

                    # Create labels for autoregressive training of decoder
                    decoder_input_ids = labels[:, :-1].contiguous()
                    labels = labels.contiguous()

                    # Add padding to the start of decoder input
                    decoder_input_ids = F.pad(decoder_input_ids, (1, 0), value=tokenizer.pad_token_id)

                    # Forward pass
                    logits, loss = model(input_ids, decoder_input_ids, labels)
                    val_loss += loss.item()

            for param_group in optimizer.param_groups:
                lr_log = param_group['lr']

            print(f"Epoch {epoch}, Step {step}, Train Loss: {total_loss/(step+1)}, Val Loss: {val_loss/val_step}, Learning Rate: {lr_log}")
            # wandb.log({"Train Loss": total_loss/(step+1), "Val Loss": val_loss/val_step, "Learning Rate": lr_log, "step": n_step})
            model.train()

Epoch 0, Step 0, Train Loss: 10.513018608093262, Val Loss: 11.196803919474284, Learning Rate: 7.8125e-06
Epoch 0, Step 100, Train Loss: 6.45186456595317, Val Loss: 4.079324197769165, Learning Rate: 0.0007890625
Epoch 0, Step 200, Train Loss: 4.822129995668705, Val Loss: 2.968639612197876, Learning Rate: 0.0015703125
Epoch 0, Step 300, Train Loss: 4.055846433702893, Val Loss: 2.5050491015116374, Learning Rate: 0.0023515625
Epoch 0, Step 400, Train Loss: 3.6193564627830526, Val Loss: 2.382992235819499, Learning Rate: 0.0031211010589932645


In [31]:
model.eval()

# Generate translations
sample = val_dataset[np.random.randint(len(val_dataset))]
input_ids = tokenizer(sample["en"], padding='max_length', max_length=config['block_size'], truncation=True, return_tensors="pt").input_ids.to(device)
dec_inp = model.generate_v1(input_ids, greedy=True)
dec_inp1 = model.generate(input_ids)

translation = tokenizer.decode(dec_inp[0], skip_special_tokens=True)
translation1 = tokenizer.decode(dec_inp1[0], skip_special_tokens=True)
print("Source:", sample["en"])
print("Target:      ", sample["de"])
print("Translation: ", translation)
print("Translation1:", translation1)

Source: Many of the chairs are empty with only a few people enjoying the sun.
Target:       Viele Stühle sind leer und nur einige wenige Menschen genießen die Sonne.
Translation:  viele personen sind sich sich mit einem gesicht sich vorbeigehen.
Translation1: viele personen sind sich sich mit einem gesicht sich vorbeigehen.


In [ ]:
wandb.finish()

Learning Rate,▁▃▅▆██▇▆▆▆▆▅▅▅▅▅▅▅▄▄▄▄▄▄▄▄▄▄▄▄▄▄
Train Loss,█▅▄▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Val Loss,█▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇███
Learning Rate,0.00118
Train Loss,0.89609
Val Loss,1.48424
step,2825
